In [1]:
import qiskit.extensions.simulator  
from qiskit import *

import os 
import sys 
import unittest
import logging
#import Qconfig
#import QconfigQobj
import QConfigCore
import QConfigStaging
#register(Qconfig.APItoken)
#register(QConfigQobj.APItoken, QConfigQobj.config['url'])
register(QConfigCore.APItoken, QConfigCore.config['url'])
register(QConfigStaging.APItoken, QConfigStaging.config['url'])
available_backends() # this is all the backends i have access to

logger = logging.Logger('error')
sys.path.append('../')

The backend names "['ibmq_qasm_simulator']" of this provider are already in use. Refer to documentation for `available_backends()` and `unregister()`.


In [2]:
available_backends()

['ibmq_1_atlantis',
 'ibmq_qasm_simulator',
 'ibmqx_hpc_qasm_simulator',
 'local_qasm_simulator',
 'local_statevector_simulator',
 'local_unitary_simulator']

In [3]:
backends = ['ibmqx_hpc_qasm_simulator','ibmq_1_atlantis','ibmq_qasm_simulator'] 

In [4]:
from qiskit.backends.ibmq import IBMQProvider
from python.common import Path, QiskitTestCase, requires_qe_access, slow_test


class TestBackendQobj(QiskitTestCase):
    """Qiskit backend qobj test."""
    
    def __init__(self, backend_name, *args, **kwargs):
        self.backend_name = backend_name
        super(TestBackendQobj, self).__init__(*args, **kwargs)
    
    def test_operational(self):
        local = get_backend('local_qasm_simulator')
        remote = get_backend(self.backend_name)
        self.assertTrue(remote.status()['operational'])
    
    def test_allow_qobj(self):
        local = get_backend('local_qasm_simulator')
        remote = get_backend(self.backend_name)
        self.assertTrue(remote.configuration()['allow_q_object'])
    def test_one_qubit_no_operation(self):
        """Test one circuit, one register, in-order readout.
        """
        qr = QuantumRegister(1)
        cr = ClassicalRegister(1)
        circ = QuantumCircuit(qr, cr)
        circ.measure(qr[0], cr[0])
        local = get_backend('local_qasm_simulator')
        remote = get_backend(self.backend_name)
        qobj = compile(circ, remote)
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)
        
    def test_one_qubit_operation(self):
        """Test one circuit, one register, in-order readout.
        """
        qr = QuantumRegister(1)
        cr = ClassicalRegister(1)
        circ = QuantumCircuit(qr, cr)
        circ.x(qr[0])
        circ.measure(qr[0], cr[0])
        local = get_backend('local_qasm_simulator')
        remote = get_backend(self.backend_name)
        qobj = compile(circ, remote)
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)
        
    def test_simple_circuit(self):
        """Test one circuit, one register, in-order readout.
        """
        qr = QuantumRegister(4)
        cr = ClassicalRegister(4)
        circ = QuantumCircuit(qr, cr)
        circ.x(qr[0])
        circ.x(qr[2])
        circ.measure(qr[0], cr[0])
        circ.measure(qr[1], cr[1])
        circ.measure(qr[2], cr[2])
        circ.measure(qr[3], cr[3])
        local = get_backend('local_qasm_simulator')
        remote = get_backend(self.backend_name)
        qobj = compile(circ, remote)
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)


    def test_readout_order(self):
        """Test one circuit, one register, out-of-order readout.
        """
        qr = QuantumRegister(4)
        cr = ClassicalRegister(4)
        circ = QuantumCircuit(qr, cr)
        #circ.x(qr[0])
        #circ.x(qr[2])
        circ.measure(qr[0], cr[2])
        circ.measure(qr[1], cr[0])
        circ.measure(qr[2], cr[1])
        circ.measure(qr[3], cr[3])
        local = get_backend('local_qasm_simulator')
        remote = get_backend(self.backend_name)
        qobj_remote = compile(circ, remote)
        qobj_local = compile(circ, local)
        result_remote = remote.run(qobj_remote).result()
        result_local = local.run(qobj_local).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)

                
    def test_multi_register(self):
        """Test one circuit, two registers, out-of-order readout.
        """
        qr1 = QuantumRegister(2)
        qr2 = QuantumRegister(2)
        cr1 = ClassicalRegister(3)
        cr2 = ClassicalRegister(1)
        circ = QuantumCircuit(qr1, qr2, cr1, cr2)
        circ.h(qr1[0])
        circ.cx(qr1[0], qr2[1])
        circ.h(qr2[0])
        circ.cx(qr2[0], qr1[1])
        circ.x(qr1[1])
        circ.measure(qr1[0], cr2[0])
        circ.measure(qr1[1], cr1[0])
        circ.measure(qr1[1], cr2[0])
        circ.measure(qr1[1], cr1[2])
        circ.measure(qr2[0], cr1[2])
        circ.measure(qr2[1], cr1[1])
        local = get_backend('local_qasm_simulator')
        remote = get_backend(self.backend_name)
        qobj = compile(circ, remote)
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)

    def test_multi_circuit(self):
        """Test two circuits, two registers, out-of-order readout.
        """
        qr1 = QuantumRegister(2)
        qr2 = QuantumRegister(2)
        cr1 = ClassicalRegister(3)
        cr2 = ClassicalRegister(1)
        circ1 = QuantumCircuit(qr1, qr2, cr1, cr2)
        circ1.h(qr1[0])
        circ1.cx(qr1[0], qr2[1])
        circ1.h(qr2[0])
        circ1.cx(qr2[0], qr1[1])
        circ1.x(qr1[1])
        circ1.measure(qr1[0], cr2[0])
        circ1.measure(qr1[1], cr1[0])
        circ1.measure(qr1[0], cr2[0])
        circ1.measure(qr1[1], cr1[2])
        circ1.measure(qr2[0], cr1[2])
        circ1.measure(qr2[1], cr1[1])
        circ2 = QuantumCircuit(qr1, qr2, cr1)
        circ2.h(qr1[0])
        circ2.cx(qr1[0], qr1[1])
        circ2.h(qr2[1])
        circ2.cx(qr2[1], qr1[1])
        circ2.measure(qr1[0], cr1[0])
        circ2.measure(qr1[1], cr1[1])
        circ2.measure(qr1[0], cr1[2])
        circ2.measure(qr2[1], cr1[2])
        local = get_backend('local_qasm_simulator')
        remote = get_backend(self.backend_name)

        qobj = compile([circ1, circ2], remote)
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ1),
                              result_local.get_counts(circ1),delta=50)
        self.assertDictAlmostEqual(result_remote.get_counts(circ2),
                              result_local.get_counts(circ2),delta=50)
    
    def test_conditional_operation(self):
        """Test conditional operation. 
        """
        qr = QuantumRegister(4)
        cr = ClassicalRegister(4)
        circ = QuantumCircuit(qr, cr)
        circ.x(qr[0])
        circ.x(qr[2])
        circ.measure(qr[0], cr[0])
        circ.x(qr[0]).c_if(cr,1)
        local = get_backend('local_qasm_simulator')
        remote = get_backend(self.backend_name)

        qobj = compile(circ, remote)
        result_remote = remote.run(qobj).result()
        result_local = local.run(qobj).result()
        self.assertDictAlmostEqual(result_remote.get_counts(circ),
                              result_local.get_counts(circ),delta=50)


QISKIT_TESTS: {'skip_online': False, 'mock_online': False, 'run_slow': False, 'rec': False, '': True}


In [5]:
def test_backend(backend):
    backend
    print("Testing: {}".format(backend))
    testbackend = TestBackendQobj(backend)
    test_methods = [testbackend.test_allow_qobj,testbackend.test_operational,testbackend.test_one_qubit_operation,
                    testbackend.test_one_qubit_no_operation,testbackend.test_multi_circuit,testbackend.test_multi_register,
               testbackend.test_readout_order,testbackend.test_simple_circuit,testbackend.test_conditional_operation]

    failed = 0 
    for test in test_methods:
        try:
            test()
            print('.',end='')
        except Exception as err:
            failed += 1
            print('x',end='')
            logger.warning(err)
            logger.error('test failed:{}'.format(str(test)))
    print('\n {0} of {1} tests failed'.format(failed,len(test_methods)))

In [6]:
#ibmq sim 
backend = backends[0]
test_backend(backend)

Testing: ibmqx_hpc_qasm_simulator
.

Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Couldn't get backend status: Expecting value: line 1 column 1 (char 0)
test failed:<bound method TestBackendQobj.test_operational of <__main__.TestBackendQobj testMethod=runTest>>


x......

Got a 400 code response to https://q-console-api-staging.mybluemix.net/api/Network/qobj-test-hub/Groups/qobj-test/Projects/normal/jobs?access_token=r0MCVWHrGmVo22rOFCHCPO4ZcOY9NuFfVIoBMYKCp6iWpEG7rVERER4xUovrpbHW: {"error":{"status":400,"message":"Error parsing QASM. Error parsing qasm number 0. Operations are disabled","code":"QASM_NOT_VALID","statusCode":400}}
Got a 400 code response to https://q-console-api-staging.mybluemix.net/api/Network/qobj-test-hub/Groups/qobj-test/Projects/normal/jobs?access_token=r0MCVWHrGmVo22rOFCHCPO4ZcOY9NuFfVIoBMYKCp6iWpEG7rVERER4xUovrpbHW: {"error":{"status":400,"message":"Error parsing QASM. Error parsing qasm number 0. Operations are disabled","code":"QASM_NOT_VALID","statusCode":400}}
Got a 400 code response to https://q-console-api-staging.mybluemix.net/api/Network/qobj-test-hub/Groups/qobj-test/Projects/normal/jobs?access_token=r0MCVWHrGmVo22rOFCHCPO4ZcOY9NuFfVIoBMYKCp6iWpEG7rVERER4xUovrpbHW: {"error":{"status":400,"message":"Error parsing QASM. Er

x
 2 of 9 tests failed


In [7]:
#normal atlantis (pinky)
backend = backends[1]
test_backend(backend)

Testing: ibmq_1_atlantis
..

('0': 1024 != 0), ('1': 0 != 1024) within 50 delta
test failed:<bound method TestBackendQobj.test_one_qubit_operation of <__main__.TestBackendQobj testMethod=runTest>>


x.

Got a 400 code response to https://q-console-api.mybluemix.net/api/Network/ibm-q-internal/Groups/development/Projects/qobj-test/jobs?access_token=e2y97EuoI8GZkhrXvm07hn7MZloBSynfq4wlEyb4vQAELAmn6XhAAP1OC780azC2: {"error":{"status":400,"message":"Error parsing QASM. Error parsing qasm number 1. Gates after a measure are blocked","code":"QASM_NOT_VALID","statusCode":400}}
Got a 400 code response to https://q-console-api.mybluemix.net/api/Network/ibm-q-internal/Groups/development/Projects/qobj-test/jobs?access_token=e2y97EuoI8GZkhrXvm07hn7MZloBSynfq4wlEyb4vQAELAmn6XhAAP1OC780azC2: {"error":{"status":400,"message":"Error parsing QASM. Error parsing qasm number 1. Gates after a measure are blocked","code":"QASM_NOT_VALID","statusCode":400}}
Got a 400 code response to https://q-console-api.mybluemix.net/api/Network/ibm-q-internal/Groups/development/Projects/qobj-test/jobs?access_token=e2y97EuoI8GZkhrXvm07hn7MZloBSynfq4wlEyb4vQAELAmn6XhAAP1OC780azC2: {"error":{"status":400,"message":"Error pa

x

...

x

('0000': 233 != 1024), ('0101': 276 != 0), ('0100': 238 != 0), ('0001': 277 != 0) within 50 delta
test failed:<bound method TestBackendQobj.test_readout_order of <__main__.TestBackendQobj testMethod=runTest>>


x

('0011': 233 != 0), ('0010': 271 != 0), ('0000': 253 != 0), ('0001': 267 != 0), ('0101': 0 != 1024) within 50 delta
test failed:<bound method TestBackendQobj.test_simple_circuit of <__main__.TestBackendQobj testMethod=runTest>>
internal error
test failed:<bound method TestBackendQobj.test_conditional_operation of <__main__.TestBackendQobj testMethod=runTest>>


xx
 6 of 9 tests failed


In [8]:
#ibmqx hpc sim  
backend = backends[2]
test_backend(backend)

False is not true
test failed:<bound method TestBackendQobj.test_allow_qobj of <__main__.TestBackendQobj testMethod=runTest>>


Testing: ibmq_qasm_simulator
x........
 1 of 9 tests failed
